# Introduction
[Chainer](http://chainer.org/) とはニューラルネットの実装を簡単にしたフレームワークです。

* 今回は機械翻訳にニューラルネットを適用してみました。

![](./pictures/Chainer.jpg)

* 今回は機械翻訳を行っていただきます。

ハンズオンについての説明
http://qiita.com/GushiSnow/private/b0abf7d3dccafe14fa07

機械翻訳は機械が言語を別の言語に翻訳するものです。

機械翻訳にはいくつか種類があるのでここでも紹介しておきます。

* PBMT(Phrase Base Machine Translation)モデル
 * [moses](http://www.statmt.org/moses/)というオープンソースで使用できるメジャーな機械翻訳のモデルですが、難しすぎて理解できない人を続出させる機械翻訳の鬼門です
* ニューラル機械翻訳
 * 翻訳元単語の辞書ベクトルを潜在空間ベクトルに落とし込み、ニューラルネットで翻訳先言語を学習させる手法

以下では、このChainerを利用しデータを準備するところから実際にNN翻訳モデルを構築し学習・評価を行うまでの手順を解説します。

1. [各種ライブラリ導入](#各種ライブラリ導入) 
2. [機械翻訳のクラス](#機械翻訳のクラス)
3. [翻訳処理を行うforwardに必要なパラメータ設定](#翻訳処理を行うforwardに必要なパラメータ設定)
4. [翻訳処理を行うEncoder処理部分](#翻訳処理を行うEncoder処理部分)
5. [翻訳処理を行うDecoder処理部分](#翻訳処理を行うDecoder処理部分)
6. [翻訳処理を行うforward処理部分](#翻訳処理を行うforward処理部分)
7. [各値を設定](#各値を設定)
8. [実行](#実行)
9. [学習したモデルを使用したテスト (Advanced)](#学習したモデルを使用したテスト (Advanced))
10. [学習したモデルを評価 (Advanced)](#学習したモデルを評価 (Advanced))

## 1.各種ライブラリ導入

Chainerの言語処理では多数のライブラリを導入します。



In [2]:
from util.functions import trace
import numpy as np

from chainer import functions, optimizers

from util.chainer_cpu_wrapper import wrapper

from EncoderDecoderModel import EncoderDecoderModel
import subprocess

`導入するライブラリの代表例は下記です。

* `numpy`: 行列計算などの複雑な計算を行なうライブラリ
* `chainer`: Chainerの導入
* `util`:今回の処理で必要なライブラリが入っています。


## 2.機械翻訳のクラス

下記を設定しています。
* ニューラルネットを用いて機械翻訳用のモデルを構成しています。

全体構成

![](./pictures/NN_machine_translation.png)




## 3.翻訳処理を行うforwardに必要なパラメータ設定

下記のコードで必要なパラメータを設定するクラスを定義しています。

In [3]:
class EncoderDecoderModelParameter():
    
    def __init__(self, is_training, src_batch, encoderDecoderModel, trg_batch = None, generation_limit = None):
        self.m = encoderDecoderModel.model
        self.tanh = functions.tanh
        self.lstm = functions.lstm
        self.batch_size = len(src_batch)
        self.src_len = len(src_batch[0])
        self.src_stoi = encoderDecoderModel.src_vocab.stoi
        self.trg_stoi = encoderDecoderModel.trg_vocab.stoi
        self.trg_itos = encoderDecoderModel.trg_vocab.itos
        self.state_c = wrapper.zeros((self.batch_size, encoderDecoderModel.n_hidden))
        self.trg_batch = trg_batch
        self.generation_limit = generation_limit


## 4.翻訳処理を行うEncoder処理部分

下記の論文を参考にしてforward処理を記述しています。

http://papers.nips.cc/paper/5346-sequence-to-sequence-learning-with-neural-networks.pdf

* Encoder部分
もっとも特徴的な部分は翻訳元言語を逆順にしていることです。そうすることで精度が向上していると述べており、今回の翻訳のNNモデルもそれを再現しています。

この論文でははっきりした要因はわかっていないが、おそらく翻訳先の言語と翻訳元言語の距離が逆順にすることで最初の単語の距離が近くなり、翻訳のタイムラグが少なくなったことが起因していると考えられています。

![](./pictures/encoder.png)



In [4]:
class EncoderDecoderModelEncoding():
    
    def encoding(self, src_batch, parameter, trg_batch = None, generation_limit = None):

#--------Hands on------------------------------------------------------------------#
    # encoding
        #翻訳元言語の末尾</s>を潜在空間に写像し、隠れ層に入力、lstmで出力までをバッチサイズ分行う。
        state_x = wrapper.make_var([parameter.src_stoi('</s>') for _ in range(parameter.batch_size)], dtype=np.int32)
        state_i = parameter.tanh(parameter.m.w_xi(state_x))
        parameter.status_c, state_p = parameter.lstm(parameter.state_c, parameter.m.w_ip(state_i))
        
        #翻訳元言語を逆順に１と同様の処理を行う。   
        for l in reversed(range(parameter.src_len)):
            #翻訳元言語を語彙空間に写像
            state_x = wrapper.make_var([parameter.src_stoi(src_batch[k][l]) for k in range(parameter.batch_size)], dtype=np.int32)
            #語彙空間を潜在空間（次元数が減る）に写像
            state_i = parameter.tanh(parameter.m.w_xi(state_x))
            #状態と出力結果をlstmにより出力。lstmの入力には前の状態と語彙空間の重み付き出力と前回の重み付き出力を入力としている
            parameter.state_c, state_p = parameter.lstm(parameter.status_c, parameter.m.w_ip(state_i) + parameter.m.w_pp(state_p))

        #次のミニバッチ処理のために最終結果をlstmで出力。翻訳の仮説用のリストを保持
        parameter.state_c, state_q = parameter.lstm(parameter.state_c, parameter.m.w_pq(state_p))
        hyp_batch = [[] for _ in range(parameter.batch_size)]
        return state_q, hyp_batch
#--------Hands on------------------------------------------------------------------#

## 5.翻訳処理を行うDecoder処理部分

* Decoder部分

学習部分と予測部分を実装しています。学習部分ではターゲット先の単語の取得と損失の計算をしています。
またlstmで次回の学習に使用する部分では学習では正解の翻訳、予測では予測した翻訳を使用しています。

![](./pictures/decorder.png)

In [5]:
class EncoderDecoderModelDecoding():
    
    def decoding(self, is_training, src_batch, parameter, s_q, hyp_batch, trg_batch = None, generation_limit = None):

#--------Hands on------------------------------------------------------------------#
    # decoding
        """
　　     学習と予測をコードしている。
        """
        if is_training:
            #損失の初期化及び答えとなる翻訳先言語の長さを取得。（翻訳元言語と翻訳先言語で長さが異なるため）
            accum_loss = wrapper.zeros(())
            trg_len = len(parameter.trg_batch[0])

            #ニューラルネットの処理は基本的にEncodingと同一であるが、損失計算と翻訳仮説候補の確保の処理が加わっている。
            for l in range(trg_len):
                #翻訳元言語からのニューラルの出力を受け取り、潜在空間に写像。
                s_j = parameter.tanh(parameter.m.w_qj(s_q))
                #潜在空間から翻訳先言語の空間に写像
                r_y = parameter.m.w_jy(s_j)
                #答えとなる翻訳結果を取得
                s_t = wrapper.make_var([parameter.trg_stoi(parameter.trg_batch[k][l]) for k in range(parameter.batch_size)], dtype=np.int32)
                #損失計算
                accum_loss += functions.softmax_cross_entropy(r_y, s_t)
                #翻訳仮説出力
                output = wrapper.get_data(r_y).argmax(1)

                #翻訳仮説確保
                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                #状態と出力結果をlstmにより出力。lstmの入力には前の状態と語彙空間の重み付き出力と前回の重み付き出力を入力としている
                parameter.status_c, s_q = parameter.lstm(parameter.status_c, parameter.m.w_yq(s_t) + parameter.m.w_qq(s_q))
            return hyp_batch, accum_loss
        else:
            """
            予測部分
            """
            #予測では予測する翻訳言語の長さに制約をしないとニューラル翻訳モデルの性質上、無限に翻訳してしまう可能性があるので、長さに制約を設けている。
            while len(hyp_batch[0]) < parameter.generation_limit:
                s_j = parameter.tanh(parameter.m.w_qj(s_q))
                r_y = parameter.m.w_jy(s_j)
                output = wrapper.get_data(r_y).argmax(1)

                #翻訳仮説確保
                for k in range(parameter.batch_size):
                    hyp_batch[k].append(parameter.trg_itos(output[k]))

                #ミニバッチサイズ分の翻訳仮説の末尾が</s>になったときにDecoding処理が終わるようになっている。
                if all(hyp_batch[k][-1] == '</s>' for k in range(parameter.batch_size)): break
                
                #次のlstmの処理のために出力結果と状態を渡している
                s_y = wrapper.make_var(output, dtype=np.int32)
                parameter.status_c, s_q = parameter.lstm(parameter.status_c, parameter.m.w_yq(s_y) + parameter.m.w_qq(s_q))

            return hyp_batch
        
#--------Hands on------------------------------------------------------------------#

## 6.翻訳処理を行うforward処理部分

上記の処理を実行するためのメソッドです。

In [6]:
class EncoderDecoderModelForward(EncoderDecoderModel):
    
    def forward(self, is_training, src_batch, trg_batch = None, generation_limit = None):
        parameter = EncoderDecoderModelParameter(is_training, src_batch, self, trg_batch, generation_limit)
        
    # encoding
        encoder = EncoderDecoderModelEncoding()
        s_q, hyp_batch = encoder.encoding(src_batch, parameter)
    # decoding
        decoder = EncoderDecoderModelDecoding()
        if is_training:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)
        else:
            return decoder.decoding(is_training, src_batch, parameter, s_q, hyp_batch, trg_batch, generation_limit)

## 7.各値を設定

各値を設定

* 翻訳元言語の設定
* 翻訳先言語の設定
* 語彙の設定
* 潜在空間の設定
* 隠れ層の設定
* 学習回数の設定
* ミニバッチサイズの設定
* 最大予測言語数の設定


In [7]:
parameter_dict = {}
train_path = "oda201512handson/train/"
test_path = "oda201512handson/test/"
#--------Hands on  2----------------------------------------------------------------
# #

parameter_dict["source"] = train_path + "train1000.ja"
parameter_dict["target"] = train_path + "train1000.en"
parameter_dict["test_source"] = test_path + "test1000.ja"
parameter_dict["test_target"] = test_path + "test1000_hyp.en"
parameter_dict["reference_target"] = test_path + "test1000.en"
parameter_dict["vocab"] = 550
parameter_dict["embed"] = 500
parameter_dict["hidden"] = 20
parameter_dict["epoch"] = 20
parameter_dict["minibatch"] = 64
parameter_dict["generation_limit"] = 256
parameter_dict["show_hands_on_number"] = 10
parameter_dict["show_i_epoch"] = 0
#--------Hands on  2----------------------------------------------------------------#

## 8.実行

In [ ]:
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.train_model()

2015-12-08 18:31:49.553484 ... initializing ...
2015-12-08 18:31:49.554355 ... making vocaburaries ...
2015-12-08 18:31:49.583882 ... making model ...
2015-12-08 18:31:49.615206 ... epoch 1/20: 
2015-12-08 18:31:49.744711 ... epoch   1/ 20, sample       24
2015-12-08 18:31:49.745071 ...   src = 彼 も それ を 見 た 。 * * * * * * *
2015-12-08 18:31:49.745364 ...   trg = he saw it also . *
2015-12-08 18:31:49.745594 ...   hyp = several isn almost not it early
2015-12-08 18:31:51.727287 ... saving model ...
2015-12-08 18:31:52.344344 ... epoch 2/20: 
2015-12-08 18:31:52.503455 ... epoch   2/ 20, sample       24
2015-12-08 18:31:52.503855 ...   src = 彼 も それ を 見 た 。 * * * * * * *
2015-12-08 18:31:52.504211 ...   trg = he saw it also . *
2015-12-08 18:31:52.504472 ...   hyp = he is <unk> is <unk> *
2015-12-08 18:31:55.248103 ... saving model ...
2015-12-08 18:31:55.802383 ... epoch 3/20: 
2015-12-08 18:31:55.913131 ... epoch   3/ 20, sample       24
2015-12-08 18:31:55.913482 ...   src = 彼 も それ を 見 

## 9.学習したモデルを使用したテスト 

学習したモデルを使用してテスト

* 学習したモデルを利用してテストデータ（日本語）を英語に翻訳しモデルに保存。

In [ ]:
model_name = "ChainerMachineTranslation.021"
trace('initializing ...')
wrapper.init()

encoderDecoderModel = EncoderDecoderModelForward(parameter_dict)
encoderDecoderModel.test_model(model_name)

## 10.学習したモデルを評価 (Advanced)

学習したモデルの評価するため、BLEUを算出

* BlEUとは翻訳の客観的評価に使用される指標で、答えとなる文章との一致率を評価する方法を用いています。
詳しく知りたい方は下記をご覧ください。
http://www2.nict.go.jp/univ-com/multi_trans/member/mutiyama/corpmt/4.pdf

In [ ]:
cmd_corpus = "mteval-corpus  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
cmd_sentence = "mteval-sentence  -e BLEU RIBES -r " +parameter_dict["reference_target"] + " -h " + parameter_dict["test_target"]
mteval_corpus = subprocess.Popen(cmd_corpus, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout_data = mteval_corpus.stdout.read()
print(stdout_data.decode('utf-8'))